In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
%matplotlib inline

# HW4

## Reading data from file

In [2]:
data = pd.read_csv("churn_analysis.csv")
data = data.drop(data.columns[0], axis=1); # column with row id

**Описание данных:**
* state — штат США
* account_length — длительность использования аккаунта
* area_code — деление пользователей на псевдорегионы, использующееся в телекоме
* intl_plan — подключена ли у пользователя услуга международного общения
* vmail_plan — подключена ли у пользователя услуга голосовых сообщений
* vmail_message — количество голосых сообщений, который пользователь отправил / принял
* day_calls — сколько пользователь совершил дневных звонков
* day_mins — сколько пользователь проговорил минут в течение дня
* day_charge — сколько пользователь заплатил за свою дневную активность
* eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности
* night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности
* intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения
* custserv_calls — сколько раз пользователь позвонил в службу поддержки
* treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)
* mes_estim — оценка интенсивности пользования интернет мессенджерами
* churn — результат оттока: перестал ли абонент пользоваться услугами оператора

In [3]:
data.describe()

,account_length,area_code,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,437.182418,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856,0.990999,0.484236
std,39.822106,42.371290,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491,0.819138,0.138560
min,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000
25%,74.000000,408.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000,0.000000,0.390000
50%,101.000000,415.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000,1.000000,0.480000
75%,127.000000,510.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000,2.000000,0.580000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000,2.000000,0.960000


## Solving tasks

### Task 1
Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Заметьте, что, например, (AZ, HI) и (HI, AZ) — это одна и та же пара. Какой критерий подходит для решения этой задачи? Сколько достигаемых уровней значимости оказались меньше, чем α=0.05?

In [4]:
# Selecting users from treatment1 group
t1_group = data.where(data['treatment'] == 1)
t1_group = t1_group.dropna(subset=['treatment'], how='all')

In [5]:
# creating crosstab matrix
t1_group_ct = pd.crosstab(t1_group['state'], t1_group['churn'])

Для решения данной задачи, необходимо проверить предположение о том, что исследуемая случайная величина подчиняется исследуемого закону. Это называется книтерием согласия. Существует ряд критериев согласия, одним из которых является критерий Пирсона, который и будет взят мной для проверки. Этот выбор обусловлен тем, что он является одним из наиболее распространенных непараметрических методов, позволяющих оценить значимость различий между фактическим количеством, и теоретическим количеством, котором можно ожидать в изучаемых группах при справедливости нулевой гипотезы

In [6]:
from scipy.stats import chi2_contingency as chi2
from itertools import combinations

In [9]:
def calc_singnif_less_alpha(ct_tables, alpha=0.05):
    combi = list(combinations(ct_tables.index, 2))
    counter = 0
    
    for combination in combi:
        val = ct_tables.loc[list(combination)]
        p_value = chi2(val, correction = False)[1]
    
        if p_value < alpha:
            counter += 1
    return counter

In [10]:
print('Number of signifficance level less than aplha = 0.05: ', calc_singnif_less_alpha(t1_group_ct))

Number of signifficance level less than aplha = 0.05:  34


В большом количестве случаев (34) гипотеза об отсутствии связи с уровнем точности в 0.05 не выполняется, а, значит, гипотеза отвергается

### Task 2
Посчитайте корреляции Пирсона и Спирмена между day_calls и mes_estim на всех данных, оцените их значимость, дайте интерпретацию результата.

#### Pearson correlation

In [11]:
sts.pearsonr(data.day_calls.values, data.mes_estim.values)

(-0.051794350587572625, 0.0027798836869756707)

#### Spearman correlation

In [12]:
sts.spearmanr(data.day_calls.values, data.mes_estim.values)

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

#### Интерпретация

В случае корреляции Пирсона наблюдается слабая отрицательная связь, говорящая о том, что меньшее количество звонков коррелирует с более интенсивным общением в интернет мессенджерах. А значение 0.001 < p_value < 0.01 говорит о том, что данная связь обладает средней силой сттастической значимости. 

В во втором случае, случае корреляции Спирмена, наблюдается иная ситуация. В данном случае наблюдается небольшая положительная связь, говорящая о том, что большее количество звонков коррелирует с более интенсивным общением в интернет мессенджерах. Значение p_value, находящееся в диапазоне 0.01 < p_value < 0.05 говорит о слабой статистической значимости.


### Task 3
Посчитайте значение коэффицента корреляции Крамера между штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1). Проверьте гипотезу об отсутствии связи между этими признаками.

Коэффициент V Крамера — мера силы взаимосвязи между двумя категориальными переменными.
$$ φ_{c}(X_1^n, X_2^n) =\sqrt{\frac{\chi ^{2}(X_1^n, X_2^n)}{n(min(K_{1}, K_{2}) - 1)}}, $$
где 
$$ φ_{c}(X_1^n, X_2^n) ∈ [0, 1] $$ 
0 — полное отсутствие взаимосвязи

1 — совпадение переменных (с точностью до переименования уровней).

In [18]:
np.sqrt(chi2(t1_group_ct, correction = False)[0] / len(t1_group) * (np.minimum(t1_group_ct.shape[1], t1_group_ct.shape[0]) - 1))

0.20039321502033319

#### Интерпретация

Полученный результат коэффициента корреляции Крамера находится в диапазоне 0.2 < coeff < 0.25, что говорит о наличии умеренной связи между признаками штат и отток пользователей. 
Гпиотеза об отсутствии связи была уже отвергнута в первом пункте. 

### Task 4
Проведите анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1). Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?